In [2]:
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import glob
import os
from os import path
import string
import numpy
import re
from __future__ import absolute_import, division, print_function
import pandas as pd
import multiprocessing
import gensim
import random
from tqdm import tqdm


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
def file_len(fname):
    with open(fname, encoding = 'utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [7]:
def Load_Sentences(start_year):
    os.chdir("D://Scriptie//Data//lines//cleaned")
    all_txt = glob.glob('*.txt')
    all_txt = [f for f in all_txt if f[6:12] == 'leaned']
    list_txt_range = [t for t in all_txt if int(t[0:4]) >= start_year and int(t[0:4]) < start_year + 5]
    
    sentences = list()
    
    # Get Size of Range
    range_size = 0
    numb_lines = 0
    
    for txt in list_txt_range:
        statinfo = os.stat(txt)
        range_size += int(statinfo.st_size)
        numb_lines += file_len(txt)
    
    range_size = round(range_size/ 1000000)
    sample_size = 801
    
    if range_size > 800:
        tot_lines_samp = round(sample_size * numb_lines / range_size)
        print('range size = ' + str(range_size) + '. Range is too long, sample is taken')
                
        for txt in list_txt_range:
            with open(txt, encoding = 'utf-8') as f:
                content = f.read().splitlines()
                
                if len(content) <= round(tot_lines_samp / 10):
                    for line in content:
                        sentences.append(line)
                
                if len(content) > round(tot_lines_samp / 10):
                    content = random.sample(content, round(tot_lines_samp / 10))
                    for line in content:
                        sentences.append(line)
    
    else:
        print('range size = ' + str(range_size) + '. Range falls within size limit. No sample necessary')        
        for txt in list_txt_range:
            with open(txt, encoding = 'utf-8') as f:
                content = f.read().splitlines()
                print(str(len(content)) + ' lines opened from: ' + txt)
                for line in content:
                        sentences.append(line)
    
    tmp = [line.split(' ') for line in sentences]
    return tmp

In [8]:
for i in list(range(1840,1915,5)):
    sentences = Load_Sentences(i)
    model = Word2Vec(sentences, size=150, window=12, iter = 5, min_count=100, workers=6)
    model.train(sentences,total_examples=len(sentences), epochs=10)
    model.wv.save_word2vec_format(str(i) +"_static_model.bin", binary=True)

range size = 352. Range falls within size limit. No sample necessary
37446 lines opened from: 1840_cleaned.txt
36674 lines opened from: 1841_cleaned.txt
37856 lines opened from: 1842_cleaned.txt
36684 lines opened from: 1843_cleaned.txt
50990 lines opened from: 1844_cleaned.txt
range size = 569. Range is too long, sample is taken
range size = 643. Range is too long, sample is taken
range size = 765. Range is too long, sample is taken
range size = 956. Range is too long, sample is taken
range size = 1057. Range is too long, sample is taken
range size = 904. Range is too long, sample is taken
range size = 1107. Range is too long, sample is taken
range size = 1451. Range is too long, sample is taken
range size = 1541. Range is too long, sample is taken
range size = 1878. Range is too long, sample is taken
range size = 2435. Range is too long, sample is taken
range size = 2677. Range is too long, sample is taken
range size = 2801. Range is too long, sample is taken
range size = 4033. Range

In [ ]:
count = 0

for i in list(range(1840,1915,5)):
    if count == 0:
        count += 1
        sentences = Load_Sentences(i)
        model = gensim.models.Word2Vec(sentences, min_count=100, workers=6, iter=1)
        model.save(str(year)+"_5yp_model.w2v")
        print("Model saved, on to the next\n")
        
    if count > 0:
        sentences = Load_Sentences(i)
        model.build_vocab(sentences, update=True)
        model.train(sentences, total_examples = model.corpus_count, start_alpha = model.alpha, end_alpha = model.min_alpha, epochs = model.iter)
        model.save(str(year)+"_5yp_model.w2v")
        print("Model saved, on to the next\n")

range size = 352. Range falls within size limit. No sample necessary
37446 lines opened from: 1840_cleaned.txt
36674 lines opened from: 1841_cleaned.txt
37856 lines opened from: 1842_cleaned.txt
36684 lines opened from: 1843_cleaned.txt
50990 lines opened from: 1844_cleaned.txt
Model saved, on to the next

range size = 352. Range falls within size limit. No sample necessary
37446 lines opened from: 1840_cleaned.txt
36674 lines opened from: 1841_cleaned.txt
37856 lines opened from: 1842_cleaned.txt
36684 lines opened from: 1843_cleaned.txt
50990 lines opened from: 1844_cleaned.txt


In [5]:
os.chdir("D://Scriptie//Data//lines//cleaned")
model = Word2Vec.load('1885_model.w2v')

In [6]:
model.most_similar(positive=['vaderland', 'buitenland'], negative=['binnenland'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('land', 0.7307055592536926),
 ('vorstenhuis', 0.6385587453842163),
 ('moederland', 0.6311611533164978),
 ('vadeiland', 0.6217848062515259),
 ('volk', 0.6143010854721069),
 ('rijksland', 0.5957609415054321),
 ('laud', 0.5936990976333618),
 ('werelddeel', 0.5899566411972046),
 ('staatsieven', 0.5874834060668945),
 ('christenvolk', 0.5852920413017273)]

In [32]:
def getw(word, n):
    tmp = model.most_similar(word)[n][0]
    tmp = " " + tmp + " "
    return tmp

In [36]:
list_mod = glob.glob('*.w2v')

for i in list(range(1815,1906,10)):
    mfn = [m for m in list_mod if int(m[0:4]) == i]
    model = Word2Vec.load(mfn[0])
    word = 'buitenlandsche'
    #print(str(i) + ' ' + getw(word,0) + getw(word,1) + getw(word,2) + getw(word,3))
    print(str(model.similarity('moederland', 'vaderland')))# + " | " + str(model.similarity('binnenlandsche', 'vaderlandsche'))) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.5114883
0.53946126
0.5609166
0.5714821
0.5344653
0.5383246
0.565197
0.61158526
0.57237357
0.59126824


In [30]:
getw('buitenland',2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'duitschland'